In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pyspark
spark = SparkSession.builder.appName("TF-TDF").getOrCreate()

In [0]:
ratings_rdd= spark.sparkContext.textFile("/FileStore/tables/ratings.dat").map(lambda x: x.split("::")).map(lambda x: (int(x[0]),int(x[1]),int(x[2])))

In [0]:
train_ratings_rdd, test_ratings_rdd=ratings_rdd.randomSplit(weights=[0.7, 0.3], seed=10)

In [0]:
from pyspark.mllib.recommendation import ALS
rank = 10
numIterations = 10
model = ALS.train(train_ratings_rdd, rank, numIterations)

In [0]:
test_data_without_ratings= test_ratings_rdd.map(lambda x : (x[0],x[1]))
predictions = model.predictAll(test_data_without_ratings).map(lambda x: ((x[0], x[1]), x[2]))
orginalAndPreds = test_ratings_rdd.map(lambda x: ((x[0], x[1]), x[2])).join(predictions)

In [0]:
MSE = orginalAndPreds.map(lambda x: (x[1][0] - x[1][1])**2).mean()

In [0]:
print(f"Mean Squared Error of ratings.dat file is {MSE}")
print(f"Accuracy of ratings.dat is {MSE*100}%")

Mean Squared Error of ratings.dat file is 0.8279526729277772
Accuracy of ratings.dat is 82.79526729277772%
